In [5]:
import torch
import data_loader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import json
from collections import defaultdict
# import autoencoder
import numpy as np
import data_loader_upper

In [6]:
torch.cuda.is_available()

False

In [9]:
trainx, devx, testx, trainy, devy, testy = data_loader_upper.load_all_classic_random_split(flatten=False)
# trainx, devx, testx, trainy, devy, testy = data_loader.load_all_subject_split(flatten=False)

Processing kevin
Processing russell
Splitting out test set
Splitting out dev and train set


In [10]:
trainx, trainy = data_loader.augment_train_set(trainx, trainy, augment_prop=4, is_flattened=False)
trainx.shape, devx.shape, testx.shape, trainy.shape, devy.shape, testy.shape

Augmenting TRAIN set with proportion 4


((4945, 100, 3), (10, 100, 3), (11, 100, 3), (4945,), (10,), (11,))

In [66]:
# def split_ypr(x):
#     return x[:,:,0],x[:,:,1], x[:,:,2]

In [67]:
# _,_,_,encoder = autoencoder.ae_denoise(*split_ypr(trainx))

In [68]:
# def encode(x, encoder):
#     y,p,r = autoencoder.ae_predict(*split_ypr(x), encoder)
#     return np.stack((y,p,r), axis=2)

# trainx = encode(trainx, encoder)
# devx = encode(devx, encoder)
# testx = encode(testx, encoder)

In [11]:
trainx.shape, devx.shape, testx.shape, trainy.shape, devy.shape, testy.shape

((4945, 100, 3), (10, 100, 3), (11, 100, 3), (4945,), (10,), (11,))

In [12]:
BATCH_SIZE = 500

def get_dataloader(x, y, batch_size):
    dataset = [(x[i].T, y[i]) for i in range(y.shape[0])]
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

trainloader = get_dataloader(trainx, trainy, BATCH_SIZE)
devloader = get_dataloader(devx, devy, BATCH_SIZE)
testloader = get_dataloader(testx, testy, BATCH_SIZE)

In [13]:
_, num_feature, num_channel = trainx.shape
num_feature, num_channel

(100, 3)

In [14]:
def acc_loss(data_loader, criterion):
    correct = 0
    total = 0
    total_loss = 0.0
    with torch.no_grad():
        for data in data_loader:
            x, y = data
            if torch.cuda.is_available():
                x = x.cuda()
                y = y.cuda()
                
            outputs = net(x.float())
            _, predicted = torch.max(outputs.data, 1)
            
            w = torch.sum((predicted - y) != 0).item()
            r = len(y) - w
            correct += r 
            total += len(y)
            
            total_loss += criterion(outputs, y.long()).item() * len(x)
    return correct / total, total_loss / total

In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv1d(
            in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1
        )  
        # 16 channel, num_feature
        self.pool1 = nn.MaxPool1d(kernel_size=3, stride=1, padding=1)
        # 16 channel, num_feature

        self.conv2 = nn.Conv1d(
            in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2
        )
        # 32 channel, num_feature
        self.pool2 = nn.MaxPool1d(kernel_size=5, stride=1, padding=2)
        # 32 channel, num_feature
        
        self.conv3 = nn.Conv1d(
            in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2
        )
        self.pool3 = nn.MaxPool1d(kernel_size=5, stride=1, padding=2)

        self.fc1 = nn.Linear(num_feature * 64, 3200)
        self.fc2 = nn.Linear(3200, 1600)
        self.fc3 = nn.Linear(1600, 500)
        self.out = nn.Linear(500, 26)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = self.pool3(F.relu(self.conv3(x)))
        x = x.view(-1, num_feature * 64)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.out(x)
        return x
net = Net()
if torch.cuda.is_available():
    net.cuda()
net

Net(
  (conv1): Conv1d(3, 16, kernel_size=(3,), stride=(1,), padding=(1,))
  (pool1): MaxPool1d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 32, kernel_size=(5,), stride=(1,), padding=(2,))
  (pool2): MaxPool1d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
  (conv3): Conv1d(32, 64, kernel_size=(5,), stride=(1,), padding=(2,))
  (pool3): MaxPool1d(kernel_size=5, stride=1, padding=2, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=6400, out_features=3200, bias=True)
  (fc2): Linear(in_features=3200, out_features=1600, bias=True)
  (fc3): Linear(in_features=1600, out_features=500, bias=True)
  (out): Linear(in_features=500, out_features=26, bias=True)
)

In [16]:
criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.00001, momentum=0.9)
# optimizer = optim.Adam(net.parameters(), lr=0.001)
optimizer = optim.AdamW(net.parameters(), weight_decay=0.01)

hist = defaultdict(list)
for epoch in range(40):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(trainloader):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        if torch.cuda.is_available():
            inputs = inputs.cuda()
            labels = labels.cuda()

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs.float())
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()
        
    trainacc, trainloss = acc_loss(trainloader, criterion)
    devacc, devloss = acc_loss(devloader, criterion)
    hist['trainacc'].append(trainacc)
    hist['trainloss'].append(trainloss)
    hist['devacc'].append(devacc)
    hist['devloss'].append(devloss)
    
    print(f'Epoch {epoch} trainacc={trainacc} devacc={devacc}')
    print(f'        trainloss={trainloss} devloss={devloss}')

print('Finished Training')

Epoch 0 trainacc=0.06430738119312437 devacc=0.0
        trainloss=3.2447866687157516 devloss=3.230260133743286
Epoch 1 trainacc=0.30738119312436807 devacc=0.3
        trainloss=2.463794242262961 devloss=2.791567087173462
Epoch 2 trainacc=0.6432760364004044 devacc=0.4
        trainloss=1.1338668741037197 devloss=1.546929121017456


KeyboardInterrupt: 

In [62]:
acc_loss(testloader, nn.CrossEntropyLoss())

(0.402, 7.734937429428101)

In [63]:
testacc, testloss = acc_loss(testloader, nn.CrossEntropyLoss())
testacc, testloss
hist['testacc'] = testacc
hist['testloss'] = testloss

In [64]:
hist

defaultdict(list,
            {'trainacc': [0.4225544128198995,
              0.7370485529777565,
              0.866802200430519,
              0.9301841664673524,
              0.9564936618033963,
              0.9714661564219086,
              0.978115283425018,
              0.9772064099497728,
              0.9848361635972256,
              0.9903850753408275,
              0.9848600813202583,
              0.9887108347285338,
              0.9709878019612533,
              0.9873953599617317,
              0.9903133221717293,
              0.9917962209997608,
              0.9873953599617317,
              0.9875388662999283,
              0.9955991389619708,
              0.9933747907199235,
              0.992298493183449,
              0.9955034680698398,
              0.9925137526907438,
              0.9952164553934465,
              0.9939248983496771,
              0.9940923224109065,
              0.995886151638364,
              0.9866539105477159,
              0.993207

In [65]:
with open('cnn_hist_subject_with_ae.json', 'w') as f:
    json.dump(hist, f)